In [24]:
from pathlib import Path
import boto3
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

path = '/Users/6mini/post/'

for category in ['1-life/', '2-tech/']:
    root_directory = Path(path + category)

    # .md 파일을 재귀적으로 모두 찾기
    md_files = root_directory.glob('**/*.md')

    # 각 Markdown 파일에 대해 처리
    for md_file in md_files:
        # 파일의 전체 경로와 폴더 이름을 출력 (예시)
        print(f'Processing file: {md_file}')
        post_path = str(md_file).replace(path, '')
        
        main_category, sub_category, slug = post_path.split('/')
        
        main_category = main_category[2:]
        sub_category = sub_category[3:]
        slug = slug[4:].replace('.md', '')
        
        print("main_category:", main_category)
        print("sub_category:", sub_category)
        print("slug:", slug)
        
        # 함수를 사용하여 프론트매터와 내용 추출
        front_matter, content = extract_front_matter_and_content_from_markdown(md_file)

        title = front_matter['title']
        description = front_matter['description']
        date = front_matter['date']
        tags = front_matter['tags']

        print(f"title:", title)
        print(f"description:", description)
        print(f"date:", date)
        print(f"tags:", tags)
        
        # AWS 인증 및 DynamoDB 설정
        dynamodb = boto3.resource('dynamodb')

        # Posts 테이블 참조
        post_table = dynamodb.Table('Posts')
        category_table = dynamodb.Table('Categories')

        # 삽입
        response = post_table.put_item(
            Item={
                'post_id': slug,
                'title': title,
                'description': description,
                'content': content,
                'date': date,
                'main_category': main_category,
                'sub_category': sub_category,
                'tags': tags
            }
        )

        # 대분류와 관련된 서브 카테고리 가져오기
        response = category_table.get_item(
            Key={
                'main_category': main_category
            }
        )

        # 대분류가 테이블에 존재하는지 확인
        if 'Item' in response:
            # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
            if sub_category not in response['Item'].get('sub_categories', []):
                # 존재하지 않으면 서브 카테고리 추가
                updated_sub_categories = response['Item']['sub_categories']
                updated_sub_categories.append(sub_category)
                category_table.update_item(
                    Key={
                        'main_category': main_category
                    },
                    UpdateExpression="SET sub_categories = :s",
                    ExpressionAttributeValues={
                        ':s': updated_sub_categories
                    }
                )
        else:
            # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
            category_table.put_item(
                Item={
                    'main_category': main_category,
                    'sub_categories': [sub_category]
                }
            )
        break
    break

Processing file: /Users/6mini/post/1-life/01-reflections/002-2021-retrospect.md
main_category: life
sub_category: reflections
slug: 2021-retrospect
title: 2021년 회고
description: 꿈꾸던 데이터 엔지니어가 되기 위해 노력한 이윤민의 좌충우돌 2021년 한 해를 돌아본다.
date: 2022-01-06
tags: ['회고(Retrospect)']


In [ ]:
if COMMIT:
    import subprocess

    if NEW:
        msg = 'post'
    else:
        msg = 'update'
        
    try:
        # 모든 변경사항 추가
        subprocess.run(f"git add '{post_path}'", check=True, shell=True)

        # 커밋
        subprocess.run(f"git commit -m '{msg}: {title}'", check=True, shell=True)

        # 푸시
        subprocess.run("git push", check=True, shell=True)
        print("Commit and push successful.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [3]:
import os

COMMIT = 1
NEW = 1
# md 경로 복사
post_path = '1-life/01-reflections/001-codestates-ai-bootcamp-review/codestates-ai-bootcamp-review.md'

main_category, sub_category, slug, md_file = post_path.split('/')

main_category = main_category[2:]
sub_category = sub_category[3:]
slug = slug[4:]

commit_path = post_path.replace('/' + md_file, '')

main_category, sub_category, slug, md_file, commit_path

('tech',
 'AWS',
 'aws-lambda-layer-pandas',
 'aws-lambda-layer-pandas.md',
 '2-tech/03-AWS/003-aws-lambda-layer-pandas')

In [4]:
# 마크다운 파일 가져오기
import yaml

def extract_front_matter_and_content_from_markdown(post_path):
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

# 함수를 사용하여 프론트매터와 내용 추출
front_matter, content = extract_front_matter_and_content_from_markdown(post_path)

title = front_matter['title']
description = front_matter['description']
date = front_matter['date']
tags = front_matter['tags']

print(f"title: {title}")
print(f"description: {description}")
print(f"date: {date}")
print(f"tags: {tags}")



# AWS 인증 및 DynamoDB 설정
dynamodb = boto3.resource('dynamodb')

# Posts 테이블 참조
post_table = dynamodb.Table('Posts')
category_table = dynamodb.Table('Categories')

# 삽입
response = post_table.put_item(
    Item={
        'post_id': slug,
        'title': title,
        'description': description,
        'content': content,
        'date': date,
        'main_category': main_category,
        'sub_category': sub_category,
        'tags': tags
    }
)

# 대분류와 관련된 서브 카테고리 가져오기
response = category_table.get_item(
    Key={
        'main_category': main_category
    }
)

# 대분류가 테이블에 존재하는지 확인
if 'Item' in response:
    # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
    if sub_category not in response['Item'].get('sub_categories', []):
        # 존재하지 않으면 서브 카테고리 추가
        updated_sub_categories = response['Item']['sub_categories']
        updated_sub_categories.append(sub_category)
        category_table.update_item(
            Key={
                'main_category': main_category
            },
            UpdateExpression="SET sub_categories = :s",
            ExpressionAttributeValues={
                ':s': updated_sub_categories
            }
        )
else:
    # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
    category_table.put_item(
        Item={
            'main_category': main_category,
            'sub_categories': [sub_category]
        }
    )



title: AWS 람다(Lambda) 튜토리얼(3): 계층(Layer) 추가 및 판다스(Pandas) 사용 방법
description: AWS Lambda의 계층(Layer) 기능과 판다스(Pandas) 라이브러리를 Lambda 함수에 통합하는 방법을 자세히 살펴본다. Keith Rozario의 'Klayers' 프로젝트를 활용하여 판다스를 쉽게 사용하는 방법과 Lambda에서의 계층 추가 및 관리 방법을 배운다. 이를 통해 Python을 사용한 데이터 처리와 분석을 효율적으로 수행할 수 있는 방법을 탐구한다. 또한, Lambda의 서버리스 아키텍처를 통한 코드 재사용성과 라이브러리 관리의 효율성을 극대화하는 방법에 대해서도 알아볼 것이다.
date: 2024-01-17
tags: ['AWS', '람다(Lambda)', '서버리스(Serverless)', '클라우드(Cloud)', '튜토리얼(Tutorial)', '계층(Layer)', '판다스(Pandas)']
[main dfacd6e] post: AWS 람다(Lambda) 튜토리얼(3): 계층(Layer) 추가 및 판다스(Pandas) 사용 방법
 1 file changed, 163 insertions(+)
 create mode 100644 2-tech/03-AWS/003-aws-lambda-layer-pandas/aws-lambda-layer-pandas.md


KeyboardInterrupt: 

remote: error: Trace: 1d737ff463138edc45bf59c125c980525cd5c2323d4b22902cb328f04d2b4ef5        
remote: error: See https://gh.io/lfs for more information.        
remote: error: File 1-life/01-reflections/006-2023-retrospect/2023-retrospect-3.gif is 104.74 MB; this exceeds GitHub's file size limit of 100.00 MB        
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
